<a href="https://colab.research.google.com/github/realnus/scikit_learn/blob/main/HyperParameter_BOHB_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bohb-hpo

  Created wheel for bohb-hpo: filename=BOHB_HPO-0.5.2-py3-none-any.whl size=6139 sha256=63d43b0a24db7b533c31e9197399874cbad0fbdd37ec3cbcf47b0616fc561e4a
  Stored in directory: /root/.cache/pip/wheels/5b/f9/fc/d07128cc4254c8ff6b7be9c028abcb1239b5f70a5e53b72566
Successfully built bohb-hpo


In [ ]:
!pip install tpot

     |████████████████████████████████| 87 kB 3.0 MB/s 
     |████████████████████████████████| 160 kB 18.5 MB/s 
     |████████████████████████████████| 166.7 MB 13 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=8719fc430eb1e4b4eb4bb5118b7a801d998e376c9b6f7287a8ea6c30960e1060
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("drive/MyDrive/VERİ/train.csv")
test_df=pd.read_csv("drive/MyDrive/VERİ/test.csv")
gender_sur=pd.read_csv("drive/MyDrive/VERİ/gender_submission.csv")

frames = [test_df, gender_sur]
test_df = pd.concat(frames, axis=1)

train_df = train_df.drop(['PassengerId', 'Cabin', 'Ticket'], axis=1)
test_df=test_df.drop(['PassengerId', 'Cabin', 'Ticket'], axis=1)

train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
test_df['Title'] = test_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

test_df['Title'] = test_df['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_df['Title'] = test_df['Title'].replace('Mlle', 'Miss')
test_df['Title'] = test_df['Title'].replace('Ms', 'Miss')
test_df['Title'] = test_df['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_df['Title'] = train_df['Title'].map(title_mapping)
train_df['Title'] = train_df['Title'].fillna(0)

test_df['Title'] = test_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].fillna(0)

train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

train_df['Sex'] = train_df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
test_df['Sex'] = test_df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df['Age']=train_df['Age'].astype(int)
test_df['Age']=test_df['Age'].astype(int)

train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

combine= [train_df, test_df]
for dataset in combine:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age']=4

train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)

for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

for dataset in combine:
    dataset.loc[ dataset['Fare'] <=8.05, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 8.05) & (dataset['Fare'] <= 15.646), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 15.646) & (dataset['Fare'] <=  33.0), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 33.0, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("Survived", axis=1)
Y_test  = test_df["Survived"]

In [ ]:
models = []
models.append(('Logistic Regression', LogisticRegression(random_state=0)))
models.append(('Naive Bayes', GaussianNB()))
models.append(('Decision Tree (CART)',DecisionTreeClassifier(random_state=0)))
models.append(('K-NN', KNeighborsClassifier()))
models.append(('SVM', SVC(random_state=0)))
models.append(('Gradient Boosting Classifier', GradientBoostingClassifier(random_state=0)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state=0)))
models.append(('BaggingClassifier', BaggingClassifier(random_state=0)))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state=0)))
models.append(("XGBootsClassifier",xgb.XGBClassifier(random_state=0,eval_metric='mlogloss',use_label_encoder =False)))


from sklearn.metrics import accuracy_score

for name, model in models:
    model = model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    y_true = Y_test
    y_pred_train= model.predict(X_train)
    y_true_train=Y_train

    print("%s -> TEST ACC: %%%.2f     TRAİN ACC: %%%.2f" % (name,accuracy_score(y_true, y_pred)*100,accuracy_score(y_pred_train,y_true_train)*100))

Logistic Regression -> TEST ACC: %93.05     TRAİN ACC: %80.06
Naive Bayes -> TEST ACC: %81.27     TRAİN ACC: %76.26
Decision Tree (CART) -> TEST ACC: %83.08     TRAİN ACC: %87.36
K-NN -> TEST ACC: %82.18     TRAİN ACC: %85.11
SVM -> TEST ACC: %92.45     TRAİN ACC: %81.74
Gradient Boosting Classifier -> TEST ACC: %79.15     TRAİN ACC: %85.53
AdaBoostClassifier -> TEST ACC: %85.80     TRAİN ACC: %80.62
BaggingClassifier -> TEST ACC: %82.48     TRAİN ACC: %87.36
RandomForestClassifier -> TEST ACC: %83.38     TRAİN ACC: %87.36
XGBootsClassifier -> TEST ACC: %82.78     TRAİN ACC: %87.22


In [ ]:
def evaluate(params,budget):
    model = RandomForestClassifier(**params)
    cv = StratifiedKFold(n_splits=4,random_state=42,shuffle=True)
    accuracy = cross_val_score(model, X_train, Y_train, cv = cv).mean()
    return (1 - np.mean(accuracy))

In [ ]:
import bohb.configspace as cs
criterion = cs.CategoricalHyperparameter("criterion", ['entropy', 'gini'])
max_features = cs.CategoricalHyperparameter('max_features',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,'auto', 'sqrt','log2'])
n_estimators = cs.CategoricalHyperparameter("n_estimators", [*range(10,1010,10)])
bootstrap = cs.CategoricalHyperparameter("bootstrap",[True,False])
class_weight = cs.CategoricalHyperparameter("class_weight", ["balanced","balanced_subsample"])
min_samples_leaf = cs.CategoricalHyperparameter("min_samples_leaf", [*range(1,21,1)])
min_samples_split = cs.CategoricalHyperparameter("min_samples_split", [*range(2,21,1)])
max_depth = cs.CategoricalHyperparameter("max_depth", [*range(5,55,5)])
random_state = cs.CategoricalHyperparameter("random_state", [*range(1,102,1)])
configspace=cs.ConfigurationSpace([criterion, max_features,n_estimators, bootstrap, class_weight, min_samples_leaf,min_samples_split,max_depth,random_state])


In [ ]:
from bohb import BOHB

acc=[]
for i in range(3):
  int_rand=np.random.randint(0,2**32-1)
  opt = BOHB(configspace, evaluate, max_budget=100, min_budget=1,random_percent=int_rand)
  logs = opt.optimize()
  params={}
  for i in logs.best['hyperparameter']:
    params[str(i.name)]=i.value
  acc.append((params,int_rand))




In [ ]:
best_acc=[]
for params,int_rand in acc:
  trainedforest = RandomForestClassifier(**params)
  trainedforest.fit(X_train,Y_train)
  Y_pred = trainedforest.predict(X_test)
  Y_true=Y_test
  Y_pred_train= trainedforest.predict(X_train)
  Y_true_train=Y_train
  acc_random_forest =accuracy_score(Y_true, Y_pred)*100
  acc_train_random=accuracy_score(Y_pred_train, Y_true_train)*100
  best_acc.append((acc_random_forest,acc_train_random,trainedforest.get_params,int_rand))

In [ ]:
for test_acc,train_acc,parameters,int_rand in best_acc:
  print("TEST ACC: ",test_acc,"","TRAİN ACC: ",train_acc,"","PARAMS: ",parameters,"","INTEGER RAND: ",int_rand)

TEST ACC:  80.96676737160121  TRAİN ACC:  86.37640449438202  PARAMS:  <bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=45, max_features=0.9,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=320,
                       n_jobs=None, oob_score=False, random_state=38, verbose=0,
                       warm_start=False)>  INTEGER RAND:  1572045871
TEST ACC:  78.24773413897282  TRAİN ACC:  86.93820224719101  PARAMS:  <bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=25, max_features=0.7,
                       max_leaf_nodes=None,